In [6]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import matplotlib as mpl
import seaborn as sb

# General utils
from tqdm import tqdm
from os import path

from measurements import get_prev_or_next_frame, get_mesh_from_bm_image, get_tissue_curvatures
from sklearn import preprocessing, model_selection, feature_selection, metrics, inspection, pipeline
from sklearn import ensemble, linear_model
from skimage import io
import pyvista as pv
from trimesh import smoothing, Trimesh, proximity
from trimesh.curvature import discrete_mean_curvature_measure,sphere_ball_intersection

import ipyvolume as ipv

%load_ext autoreload
%autoreload 2

## Visualize 'average' cells

In [7]:
dirnames = {'R1':'/Users/xies/OneDrive - Stanford/Skin/Mesa et al/W-R1/',
           'R2':'/Users/xies/OneDrive - Stanford/Skin/Mesa et al/W-R2/'}

region = 'R1'
dirname = dirnames[region]

dataset_dir = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Mesa et al/Lineage models/Dataset pickles'
figure_dir = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Mesa et al/Lineage models/Morphology differences'
all_df = pd.read_pickle(path.join(dataset_dir,f'all_df.pkl'))
births = pd.read_pickle(path.join(dataset_dir,f'births.pkl'))

regions = {name:g for name,g in all_df.groupby('Region')}

In [8]:
# Load images
from skimage import io

tracked_cyto_by_region = {name: io.imread(path.join(dirname,'Mastodon/tracked_cyto.tif')) for name,dirname in dirnames.items()}
tracked_nuc_by_region = {name: io.imread(path.join(dirname,'Mastodon/tracked_nuc.tif')) for name,dirname in dirnames.items()}
adjdict_by_region = {name: [np.load(path.join(dirname,f'Mastodon/basal_connectivity_3d/adjacenct_trackIDs_t{t}.npy'),allow_pickle=True).item() for t in range(15)] for name,dirname in dirnames.items()}

# %loadext autoreload
%autoreload 2

In [26]:
from imageUtils import trim_multimasks_to_shared_bounding_box, pad_image_to_size_centered, create_average_object_from_multiple_masks
from visualization import extract_nuc_and_cell_and_microenvironment_mask_from_idx,get_nuc_and_cell_and_microenvironment_movie

standard_size = (40,150,150)

def get_display_of_average_nuc_cyto_micro(ncm_list,largest_size=None):
    nuc = [m[0] for m in ncm_list]
    cyto = [m[1] for m in ncm_list]
    micro = [m[2] for m in ncm_list]
    nuc = create_average_object_from_multiple_masks(nuc,prealign=False)
    cyto = create_average_object_from_multiple_masks(cyto,prealign=False)
    micro = create_average_object_from_multiple_masks(micro,prealign=False)

    if largest_size is None:
        largest_size = np.array((nuc.shape,cyto.shape,micro.shape)).max(axis=0)
    nuc = pad_image_to_size_centered(nuc,largest_size)
    cyto = pad_image_to_size_centered(cyto,largest_size)
    micro = pad_image_to_size_centered(micro,largest_size)
    return nuc,cyto,micro
    

### Visualize using mask averaging

In [10]:
df = regions[region]

In [11]:
lineageID = 196

df.droplevel(axis=1,level=1).reset_index().set_index('LineageID').loc[196][['Frame','TrackID','Cell type']]

Name,Frame,TrackID,Cell type
LineageID,,,
196.0,0,R1_608,Basal
196.0,1,R1_609,Basal
196.0,1,R1_612,Basal
196.0,2,R1_609,Basal
196.0,2,R1_612,Basal
196.0,3,R1_609,Basal
196.0,3,R1_612,Basal
196.0,4,R1_609,Basal
196.0,4,R1_612,Basal


In [22]:
[n.shape for n in nuc_movie]

[(35, 121, 121),
 (35, 121, 121),
 (35, 121, 121),
 (35, 121, 121),
 (35, 121, 121),
 (35, 121, 121),
 (35, 121, 121),
 (35, 121, 121)]

In [24]:
trackID = 609
trackID_str = f'{region}_{trackID}'

nuc_movie, cell_movie, micro_movie = get_nuc_and_cell_and_microenvironment_movie(trackID_str,df,
                                                adjdict_by_region,
                                                tracked_nuc_by_region,
                                                tracked_cyto_by_region,
                                                standard_size)

from os import makedirs
track_dir = path.join(dirname,f'Examples for figures/Microenvironment movies/{trackID}')
makedirs(track_dir,exist_ok=True)
io.imsave(path.join(track_dir,'nuclei.tif'),np.stack(nuc_movie))
io.imsave(path.join(track_dir,'cell.tif'),np.stack(cell_movie))
io.imsave(path.join(track_dir,'micro.tif'),np.stack(micro_movie))


(1, 'R1_609')
(2, 'R1_609')
(3, 'R1_609')
(4, 'R1_609')
(5, 'R1_609')
(6, 'R1_609')


/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_88156/1342370759.py:13: UserWarning: /Users/xies/OneDrive - Stanford/Skin/Mesa et al/W-R1/Examples for figures/Microenvironment movies/609/nuclei.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(path.join(track_dir,'nuclei.tif'),np.stack(nuc_movie))
/opt/miniconda3/envs/jupyter/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /Users/xies/OneDrive - Stanford/Skin/Mesa et al/W-R1/Examples for figures/Microenvironment movies/609/nuclei.tif is a low contrast image
  return func(*args, **kwargs)
/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_88156/1342370759.py:14: UserWarning: /Users/xies/OneDrive - Stanford/Skin/Mesa et al/W-R1/Examples for figures/Microenvironment movies/609/cell.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_uby

In [72]:
diff_nuc,diff_cyto,diff_micro = get_display_of_average_nuc_cyto_micro(born_diff_masks)
divide_nuc,divide_cyto,divide_micro = get_display_of_average_nuc_cyto_micro(born_divide_masks)

NameError: name 'born_diff_masks' is not defined

In [107]:
feature_names = births.columns
cyto_coeffs = [f[0] for f in feature_names if f[0].startswith('cyto_shcoeffs_L')]
nuc_coeffs = [f[0] for f in feature_names if f[0].startswith('nuc_shcoeffs_L')]

mean_diff_coeffs = born_diff[cyto_coeffs].mean()
mean_divide_coeffs = born_divide[cyto_coeffs].mean()


In [108]:
from aicsshparam import shtools

def convert_sparse_coeff_matrix(coeffdict,lmax=5):
    mat = np.zeros((2, lmax + 1, lmax + 1), dtype=np.float32)
    for l in range(lmax):
        for m in range(lmax+1):
            for c,s in enumerate(['C','S']):
                key = f'cyto_shcoeffs_L{l}M{m}{s}'
                if key in coeffdict.index:
                    mat[c,l,m] = coeffdict[key]
    return mat

mat = convert_sparse_coeff_matrix(mean_divide_coeffs)
divide_cyto_mesh = shtools.get_reconstruction_from_coeffs(mat)[0]
mat = convert_sparse_coeff_matrix(mean_diff_coeffs)
diff_cyto_mesh = shtools.get_reconstruction_from_coeffs(mat)[0]
